In [1]:
import os
import numpy as np
from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='darkgrid')

In [2]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bananapple","key":"75eb3c0872d58bbdaf9df31824376852"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d chetankv/dogs-cats-images

 98% 426M/435M [00:06<00:00, 47.3MB/s]
100% 435M/435M [00:06<00:00, 67.9MB/s]


In [5]:
#압축풀기
import zipfile

local_zip = '/content/dogs-cats-images.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/content/cat and dog')
zip_ref.close()

In [6]:
#기본경로
base_dir='/content/cat and dog/dog vs cat/dataset'

train_dir=os.path.join(base_dir,'training_set')
test_dir=os.path.join(base_dir,'test_set')

In [7]:
train_dog_dir=os.path.join(train_dir,'dogs')
train_cat_dir=os.path.join(train_dir,'cats')

test_dog_dir=os.path.join(test_dir,'dogs')
test_cat_dir=os.path.join(test_dir,'cats')

In [8]:
train_dog_list=os.listdir(train_dog_dir)
train_cat_list=os.listdir(train_cat_dir)

test_dog_list=os.listdir(test_dog_dir)
test_cat_list=os.listdir(test_cat_dir)

In [9]:
print('train dog {}' .format(len(train_dog_list)))
print('train cat {}' .format(len(train_cat_list)))

print('train dog {}' .format(len(test_dog_list)))
print('train cat {}' .format(len(test_cat_list)))

train dog 4000
train cat 4000
train dog 1000
train cat 1000


# 데이터셋/전처리

In [10]:
batch=32
img_width=250
img_height=250

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=30,
                                 shear_range=0.3,
                                 zoom_range=0.3
                                 )

test_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                  batch_size=batch,
                                                  class_mode='binary',
                                                  target_size=(img_height,img_width)
                                                  )
test_generator=test_datagen.flow_from_directory(test_dir,
                                                batch_size=batch,
                                                class_mode='binary',
                                                target_size=(img_height,img_width)
                                                )

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
print(train_generator.class_indices.items())

dict_items([('cats', 0), ('dogs', 1)])


# 모델만들기

In [16]:
import tensorflow as tf

tf.__version__

from tensorflow.keras import datasets, layers, models
from keras.preprocessing import image
from keras.engine.sequential import Sequential
from tensorflow import keras
from keras import regularizers
from tensorflow.keras.layers import Dense, Dropout, Input

In [17]:
kernel_s=(3,3)

model=models.Sequential()

model.add(layers.Conv2D(32,kernel_s,activation='relu',input_shape=(img_height,img_width,3),
                        kernel_regularizer=regularizers.l2(0.001),padding="VALID"))
model.add(layers.MaxPooling2D((2,2)))
model.add(Dropout(0.3))

model.add(layers.Conv2D(64, kernel_s, activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(Dropout(0.2))
model.add(layers.Conv2D(128, kernel_s, activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(Dropout(0.15))


model.add(layers.Flatten())
model.add(layers.Dense(32,activation='relu'))
model.add(Dropout(0.15))
model.add(layers.Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 248, 248, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 124, 124, 32)     0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 124, 124, 32)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 61, 61, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 61, 61, 64)       

In [18]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics='accuracy'
              )

# 학습하기

In [19]:
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

earlystopping=EarlyStopping(monitor='val_loss', #val_loss 값이 개선되었을 때
                            patience=5 #5회 epoch 동안 val_loss가 개선되지 않는다면 종료
                            )

In [24]:
history = model.fit(train_generator,
                    epochs=100,
                    steps_per_epoch=70,
                    validation_steps=70,
                    validation_data=test_generator,
                    callbacks=earlystopping
                    )

Epoch 1/100
70/70 [==============================] - ETA: 0s - loss: 0.5399 - accuracy: 0.7384

70/70 [==============================] - 42s 606ms/step - loss: 0.5399 - accuracy: 0.7384 - val_loss: 0.5521 - val_accuracy: 0.7280


KeyboardInterrupt: ignored